<a href="https://colab.research.google.com/github/javier-jaime/Tool-Crib/blob/master/Colab/Synthetic_Data_for_training_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Synthetic Data for training Embeddings

Generate synthetic data for training embeddings based on Microsoft's White Paper: "Improving Text Embedding with Large Language Models"
 https://arxiv.org/abs/2401.00368

In [1]:
# install libraries
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the p

In [2]:
#import libraries

import openai
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

In [ ]:
# Generate list of high-level tasks with GPT-4
# Don't forget to enter your API key

llm = ChatOpenAI(api_key="ENTER YOUR KEY HERE",
                 model_name="gpt-4")
messages = [
    HumanMessage(
        content=
"""
Brainstorm 10 potentially useful text retrieval tasks.
Here are a few examples for your reference:
- Provided a scientific claim as query, retrieve documents that help verify or refute the claim.
- Search for documents that answers a FAQ-style query on children's nutrition.
Please adhere to the following guidelines:
- Specify what the query is, and what the desired documents are.
- Each retrieval task should cover a wide range of queries, and should not be too specific.
Your output should always be a python list of strings only, with exactly 10 elements, and each element corresponds to a distinct retrieval task in one sentence. Do not explain yourself or output anything else. Be creative!
"""
    )
]
result = llm(messages)
tasks = eval(result.content)
tasks

In [ ]:
# generate one example task in JSON format

# you can change this template based based on your requirements (length of prompt, style of prompt, length of output documents, language, etc.)
template = (
    """
    You have been assigned a retrieval task: {task}
Your mission is to write one text retrieval example for this task in JSON format. The JSON object must contain the following keys:
- "user_query": a string, a random user search query specified by the retrieval task.
- "positive_document": a string, a relevant document for the user query.
- "hard_negative_document": a string, a hard negative document that only appears relevant to the query.
Please adhere to the following guidelines:
- The "user_query" should be 10-20 words, clear, and diverse in topic.
- All documents should be at least 150 words long.
- Both the query and documents should be in English.
Your output must always be a JSON object only, do not explain yourself or output anything else. Be creative!
    """
)

prompt = ChatPromptTemplate.from_messages([
    (HumanMessagePromptTemplate.from_template(template))
])

# you can put this part in a loop to create more examples
result = llm(prompt.format_prompt(task=tasks[0]).to_messages())
result.content


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'{\n  "user_query": "What are the legal implications and regulations regarding the formation of a Limited Liability Company in California?",\n  "positive_document": "The formation of a Limited Liability Company (LLC) in California requires compliance with certain laws and regulations. First, you need to file the Articles of Organization (Form LLC-1) with the Secretary of State. The document must include the name of the LLC, the purpose of the LLC, and the name and address of the registered agent. The LLC name must be unique and not confusingly similar to any existing name on record with the California Secretary of State. The registered agent must be a resident of California. An initial Statement of Information (Form LLC-12) must be filed within 90 days after filing the Articles of Organization. The filing fee for the Articles of Organization is $70 and for the Statement of Information is $20. The LLC is also subject to an $800 annual tax by the Franchise Tax Board.",\n  "hard_negative_